# Installing Z3 and Imports

In [1]:
    '''
!pip install z3-solver
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install anchor-exp
'''

'\n!pip install z3-solver\n!pip install pandas\n!pip install numpy\n!pip install sklearn\n!pip install anchor-exp\n'

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from z3 import *
import time

# Training Linear and Polynomial SVMs

## Data Preprocessing.

In [3]:
df = pd.read_csv('../datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])

In [4]:
df['target']=np.where(df['target']=='AB',1,0)

In [5]:
col_names = df.columns
col_names

Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'target'],
      dtype='object')

In [6]:
df

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,target
0,63.03,22.55,39.61,40.48,98.67,-0.25,1
1,39.06,10.06,25.02,29.00,114.41,4.56,1
2,68.83,22.22,50.09,46.61,105.99,-3.53,1
3,69.30,24.65,44.31,44.64,101.87,11.21,1
4,49.71,9.65,28.32,40.06,108.17,7.92,1
...,...,...,...,...,...,...,...
305,47.90,13.62,36.00,34.29,117.45,-4.25,0
306,53.94,20.72,29.22,33.22,114.37,-0.42,0
307,61.45,22.69,46.17,38.75,125.67,-2.71,0
308,45.25,8.69,41.58,36.56,118.55,0.21,0


In [7]:
normalized_df=(df.values[:,:-1]-df.values[:,:-1].min())/(df.values[:,:-1].max()-df.values[:,:-1].min())

In [8]:
normalized_df.min(),normalized_df.max()

(0.0, 1.0)

In [9]:
def check_targets(original_set):
    original_unique = np.unique(original_set)
    print("Original Targets: ",original_unique,"\nDesired Targets: [-1,1]")
    print("Is original the desired [1,-1]? ", np.array_equiv(original_unique,np.array([-1,1])))
    if not np.array_equiv(original_unique,np.array([-1,1])):
        if 1 in original_unique:
            print("1 exists in dataset")
            new = np.select([original_set == original_unique[0]],[-1],original_set)
        elif -1 in original_unique:
            print("-1 exists in dataset")
            new = np.select([original_set == original_unique[1]],[1],original_set)
        else:
            print("Neither exists in dataset")
            new = np.select([original_set == original_unique[0],original_set == original_unique[1]],[-1,1],original_set)
        #indexes = original_set[np.where(original_set == unique_elems[0])]
        print("New datasets consists of: ",np.unique(new))
        return new

In [10]:
targets = check_targets(df['target'])

Original Targets:  [0 1] 
Desired Targets: [-1,1]
Is original the desired [1,-1]?  False
1 exists in dataset
New datasets consists of:  [-1  1]


## Data Separation and Training

In [11]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df, targets, test_size=0.3,random_state=107) # 70% training and 30% test

In [12]:
def create_linear_classifier(kernel_type='linear'):
    return svm.SVC(kernel=kernel_type)
def create_poly_classifier(kernel_type='poly',my_degree=2,my_gamma=1/6):
    return svm.SVC(kernel=kernel_type, degree = my_degree,gamma=my_gamma)

In [13]:
clf = create_linear_classifier()
#poly = create_poly_classifier('poly',2,1/(X_train.var() * len(X_train[0])))

#Train the models using the training sets
clf.fit(X_train, y_train)
#poly.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
#poly_y_pred = poly.predict(X_test)
print("Accuracy Linear:", metrics.accuracy_score(y_test, y_pred))
#print("Accuracy Poly:", metrics.accuracy_score(y_test, poly_y_pred))

Accuracy Linear: 0.6559139784946236


In [14]:
y_pred_train = clf.predict(X_train)
#poly_pred_train = poly.predict(X_train)
print("Accuracy on Training:", metrics.accuracy_score(y_train, y_pred_train))
#print("Accuracy on Training:", metrics.accuracy_score(y_train, poly_pred_train))

Accuracy on Training: 0.6866359447004609


## SVM Decision Function For The First Element of Training Dataset

In [15]:
#Sum (coef @ sup_vec @ X[index] + bias)
((clf.dual_coef_ @ clf.support_vectors_) @ X_train[0].reshape(1, len(X_train[0])).T + clf.intercept_)[0][0]

1.2768692235824575

In [16]:
#(poly.dual_coef_ @ (((poly.support_vectors_ @ X_train[0].reshape(1, len(X_train[0])).T) * poly.gamma + poly.coef0) ** poly.degree) + poly.intercept_)[0][0]

In [17]:
#Linear SVM Decision Function
print("DualCoef / Support Vectors / X_Train.T Reshaped / Intercept (bias)")
clf.dual_coef_.shape, clf.support_vectors_.shape, X_train[0].reshape(1, len(X_train[0])).T.shape, clf.intercept_.shape

DualCoef / Support Vectors / X_Train.T Reshaped / Intercept (bias)


((1, 136), (136, 6), (6, 1), (1,))

In [18]:
#Polynomial SVM Decision Function
#print("DualCoef / Support Vectors / X_Train.T Reshaped / Gamma / Coef0 / Degree / Intercept (bias)")
#poly.dual_coef_.shape, poly.support_vectors_.shape, X_train[0].reshape(1, len(X_train[0])).T.shape,poly.gamma, poly.coef0, poly.degree, poly.intercept_

# Defining Thresholds and Finding Rejecteds

In [19]:
def limits(classifier,data):
    dec_fun = classifier.decision_function(data)
    lim_pos = dec_fun[np.argmax(dec_fun)]
    lim_neg = dec_fun[np.argmin(dec_fun)]
    return dec_fun, lim_pos, lim_neg

In [20]:
def find_thresholds(decfun,t1,t2,wr,chosen_min='EWRR'):
    solution = {'WR':0,'T1':0, 'T2':0,'E':0,'R':0,'EWRR':0}
    index = None
    n_elements = decfun.shape[0]
    for i,wr_ in enumerate(wr):
      for j in range(0,len(t1)):
        #Get Number of Rejected
        positive_indexes = np.where(decfun >= t1[j])
        negative_indexes = np.where(decfun < t2[j])
        rejected_indexes = np.where((decfun < t1[j]) & (decfun >= t2[j]))
        R = rejected_indexes[0].shape[0]
        #np.array(positive_indexes).shape,np.array(negative_indexes).shape, R

        #Get Number of Misclassifications
        class_p = y_train[positive_indexes]
        class_n = y_train[negative_indexes]
        error_p = np.where(class_p == np.unique(y_train)[0])[0].shape[0]
        error_n = np.where(class_n == np.unique(y_train)[1])[0].shape[0]
        E = (error_p + error_n)/(n_elements - R)
        R2 = R/n_elements
        #print("T1 ",round(t1[j],4),"T2 ",round(t2[j],4),"E",round(E,4),"Rej",R,"R/Total: ",R2,"Wr: ",wr_, "EwrR: ",E + wr_*R2)
        if chosen_min=='R':
            if (i == 0 and i == j) or R < solution['R']:
                solution['WR'] = wr_
                solution['T1'] = t1[j]
                solution['T2'] = t2[j]
                solution['E'] = E
                solution['R'] = R
                solution['EWRR'] = E + wr_ * R2
        elif chosen_min=='E':
            if (i == 0 and i == j) or E < solution['E']:
                solution['WR'] = wr_
                solution['T1'] = t1[j]
                solution['T2'] = t2[j]
                solution['E'] = E
                solution['R'] = R
                solution['EWRR'] = E + wr_ * R2
        elif chosen_min=='EWRR':
            if (i == 0 and i == j) or (E + wr_ * R2) < solution['EWRR']:
                solution['WR'] = wr_
                solution['T1'] = t1[j]
                solution['T2'] = t2[j]
                solution['E'] = E
                solution['R'] = R
                solution['EWRR'] = E + wr_ * R2
        else:
            return 'Chosen option "' +chosen_min+'" is invalid'
    print('Thresholds by min(',chosen_min,') from solution: ',solution)
    return solution['T1'], solution['T2']      
                

In [21]:
def find_indexes(decfun,t1,t2):
    positive_indexes = np.where(decfun >= t1)[0]
    negative_indexes = np.where(decfun < t2)[0]
    rejected_indexes = np.where((decfun < t1) & (decfun >= t2))[0]
    R = rejected_indexes.shape[0]
    return positive_indexes,negative_indexes,rejected_indexes

In [22]:
def find_thresholds_and_indexes(classifier,data,wr = None):  
    dec_fun,lim_pos,lim_neg = limits(classifier,data)
    print("Superior Limit: ",lim_pos,"\nInferior Limit: ",lim_neg)
    if wr == None:
        wr = [0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.4, 0.44, 0.48]
    t1 = []
    t2 = []
    for i in range (1,101):
      t1.append(0.01*i*lim_pos)
      t2.append(0.01*i*lim_neg)  
    T1,T2 = find_thresholds(dec_fun,t1,t2,wr)
    pos_idx,neg_idx,rej_idx = find_indexes(dec_fun,T1,T2)
    return T1, T2, pos_idx, neg_idx, rej_idx,lim_pos,lim_neg

# Implementing SVM function for Z3 Solver

## Z3 Decision Function Elements

In [23]:
np.RealVal = np.vectorize(RealVal) 
np.RealVector = np.vectorize(RealVector) 

In [24]:
def to_z3_conversion(classifier,training_set):
    z3_dual_coef = np.RealVal(classifier.dual_coef_)
    z3_support_vectors = np.RealVal(classifier.support_vectors_)
    z3_intercept_ = np.RealVal(classifier.intercept_)
    z3_X_Train = np.RealVector('x',training_set.shape[1])
    if classifier.kernel == 'poly':
        z3_gamma = np.RealVal(classifier.gamma)
        z3_coef0 = np.RealVal(classifier.coef0)
        z3_degree = np.RealVal(classifier.degree)
        return z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train, z3_gamma,z3_coef0,z3_degree
    return z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train

# Z3 with Reject Option

## Explaining the Classifier's Decision Function and Finding Relevant Features

In [25]:
def z3_explanation(classifier,t1, t2, X, z3_coef, z3_sup_vec, z3_X, z3_intercept, reject_indexes,sup_lim,inf_lim,
                   z3_gamma = None, z3_coef0 = None, z3_degree = None, show_values=True, min=0,max=1,positive=False,negative=False,rejected=False):
    elapsed_time = []
    relevant = []
    irrelevant = []
    global_values = []
    print("Number of Instances: ", len(reject_indexes))
    solver = Solver()
    if classifier.kernel=='linear':
        print("Classifier: Linear")
        if rejected:
            print("Declared: Rejected Instances")
            solver.add(Or(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t1,
                          ((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t2))
        elif positive:
            print("Declared: Positive Instances")
            solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t1)
        elif negative:
            print("Declared: Negative Instances")
            solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t2)
        else:
            print("WARNING: Must declare if are positive,negative or rejected instances!")
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= inf_lim)
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] <= sup_lim)
    elif classifier.kernel=='poly':
        print("Classifier: Polynomial")
        solver.add(Or(((z3_dual_coef @ (((z3_support_vectors @ z3_X.reshape(1, len(z3_X)).T) * z3_gamma + z3_coef0) ** z3_degree) + z3_intercept_)[0][0] >= t1),
                      (z3_dual_coef @ (((z3_support_vectors @ z3_X.reshape(1, len(z3_X)).T) * z3_gamma + z3_coef0) ** z3_degree) + z3_intercept_)[0][0] < t2))
    for j in range(0, len(z3_X)):
        solver.add(z3_X[j] >= min)
        solver.add(z3_X[j] <= max)
    solver.push()
    for i in range(0, len(reject_indexes)):
        # Add Assertions for 0<= feature <= 1
        index_list = list(range(len(z3_X)))
        unsat_list = []
        sat_list = []
        values = []

        # Select a feature and unfix it
        start = time.perf_counter()
        for z in range(0, len(z3_X)):
            
            for j in range(0, len(z3_X)):
                if j != z and j in index_list:  # Choose one to check influence
                    solver.add(z3_X[j] == X[reject_indexes[i]][j])            
            check = solver.check()
            if check == sat:
                model = solver.model()
                value = model[z3_X[z]].numerator_as_long() / model[z3_X[z]].denominator_as_long()
                sat_list.append(z)
                values.append(value)
                if show_values:
                    print('i = ', i, z, check, X[reject_indexes[i]][z], value)
            else:
                unsat_list.append(z)
                index_list.remove(z)
                if show_values:
                    print('i = ', i, z, check)
            
            solver.pop()
            solver.push()
        elapsed_time.append(time.perf_counter()  - start)    
        print("Finished ", i)
        relevant.append(sat_list)
        irrelevant.append(unsat_list)
        global_values.append(values)
        # print("Relevant: ",sat_list, '\nUnsat List: ',unsat_list,'\n')      
    for i in range(0, len(relevant)):
        if (show_values):
            print('Instance ', i, '\nRelevant Features: ', relevant[i], '\nValues: ', global_values[i], '\nIrrelevant Features: ',
                  irrelevant[i], '\nElapsed time: ',elapsed_time[i],'seconds\n\n')
    
    print("Tamanho médio de explicação: ",sum(len(x) for x in relevant)/len(relevant)," - Custo médio: ",round(sum(elapsed_time)/len(elapsed_time),5),"seg(s)")        
    return relevant, irrelevant, elapsed_time

### For Linear Classifier

#### Get thresholds and the rejected for Linear

In [26]:
T1,T2, positive_indexes,negative_indexes,rejected_indexes,lim_pos,lim_neg = find_thresholds_and_indexes(clf,X_train)
T1,T2, positive_indexes.shape[0],negative_indexes.shape[0],rejected_indexes.shape[0]

Superior Limit:  1.4216456569456146 
Inferior Limit:  0.7510107954487126
Thresholds by min( EWRR ) from solution:  {'WR': 0.04, 'T1': 0.9951519598619303, 'T2': 0.5257075568140989, 'E': 0.0, 'R': 134, 'EWRR': 0.02470046082949309}


(0.9951519598619303, 0.5257075568140989, 83, 0, 134)

#### Get Z3's equivalent to linear classifier's decision function

In [27]:
z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train = to_z3_conversion(clf,X_train)

In [28]:
rejected_linear_relevant = []
rejected_linear_irrelevant = []
positive_linear_relevant = []
positive_linear_irrelevant = []
negative_linear_relevant = []
negative_linear_irrelevant = []
rejected_elapsed_time = []
positive_elapsed_time = []
negative_elapsed_time = []

In [60]:
if len(rejected_indexes)!=0:
    if len(rejected_indexes)>=50:
        rejected_linear_relevant, rejected_linear_irrelevant, rejected_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes[0:50],
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=True, rejected = True)
    else:
        rejected_linear_relevant, rejected_linear_irrelevant, rejected_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes,
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=True, rejected = True)

Number of Instances:  50
Classifier: Linear
Declared: Rejected Instances
i =  0 0 sat 0.11431564245810055 0.5783415609688811
i =  0 1 sat 0.04506517690875232 0.37859424129326147
i =  0 2 unsat
i =  0 3 unsat
i =  0 4 sat 0.31391992551210424 0.18699682561882983
i =  0 5 sat 0.03479981378026071 0.10469635248795221
Finished  0
i =  1 0 sat 0.11538640595903166 0.5173169475483017
i =  1 1 sat 0.06522346368715085 0.3541200794496039
i =  1 2 unsat
i =  1 3 unsat
i =  1 4 sat 0.32283519553072626 0.2145028169049934
i =  1 5 sat 0.04434357541899441 0.10400220574142534
Finished  1
i =  2 0 sat 0.10991620111731842 0.7664043852603094
i =  2 1 sat 0.023859404096834264 0.4957250501784338
i =  2 2 unsat
i =  2 3 unsat
i =  2 4 sat 0.34217877094972066 0.1498716358967073
i =  2 5 sat 0.020879888268156423 0.12678340874867433
Finished  2
i =  3 0 sat 0.13067970204841714 0.8807551095569446
i =  3 1 sat 0.054399441340782125 0.5935330088945877
i =  3 2 unsat
i =  3 3 unsat
i =  3 4 sat 0.36999534450651766 0.

In [61]:
if len(positive_indexes)!=0:
    if len(positive_indexes)>=50:
        positive_linear_relevant, positive_linear_irrelevant, positive_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,positive_indexes[0:50],
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=True, positive = True)
    else:
        positive_linear_relevant, positive_linear_irrelevant, positive_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,positive_indexes,
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=True, positive = True)

Number of Instances:  50
Classifier: Linear
Declared: Positive Instances
i =  0 0 unsat
i =  0 1 unsat
i =  0 2 unsat
i =  0 3 unsat
i =  0 4 sat 0.31808659217877094 0.6337622647410798
i =  0 5 sat 0.25919459962756053 0.08535203759683944
Finished  0
i =  1 0 unsat
i =  1 1 unsat
i =  1 2 unsat
i =  1 3 sat 0.10370111731843576 0.6964516151957416
i =  1 4 sat 0.31708566108007447 0.6187300992489331
i =  1 5 sat 0.18349627560521414 0.0173807129837665
Finished  1
i =  2 0 unsat
i =  2 1 unsat
i =  2 2 unsat
i =  2 3 sat 0.20544692737430167 0.6776156781083685
i =  2 4 sat 0.30663407821229044 0.5892353360682199
i =  2 5 sat 0.17490689013035382 0.019278405046245232
Finished  2
i =  3 0 sat 0.19997672253258844 0.0
i =  3 1 sat 0.05812383612662942 0.01875726419970799
i =  3 2 unsat
i =  3 3 sat 0.16762104283054005 0.6051629699145294
i =  3 4 sat 0.29392458100558655 0.5618997543444701
i =  3 5 sat 0.1029562383612663 0.04987197585484547
Finished  3
i =  4 0 sat 0.185451582867784 0.0694514711501877

In [62]:
if len(negative_indexes)!=0:
    if len(negative_indexes)>=50:
        negative_linear_relevant, negative_linear_irrelevant, negative_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,negative_indexes[0:50],
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=True, negative = True)
    else:
        negative_linear_relevant, negative_linear_irrelevant, negative_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,negative_indexes,
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=True, negative = True)

### For Polynomial Classifier (WIP)

#### Get thresholds and the rejected for Poly

In [33]:
#T1,T2, positive_indexes,negative_indexes,rejected_indexes = find_thresholds_and_indexes(poly,X_train)
#T1,T2, positive_indexes.shape[0],negative_indexes.shape[0],rejected_indexes.shape[0]

#### Get Z3's equivalent to Poly classifier's decision function

In [34]:
#z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train,z3_gamma,z3_coef0,z3_degree = to_z3_conversion(poly,X_train)

In [35]:
#poly_relevant, poly_irrelevant = z3_explanation(poly,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes[0:1], z3_gamma,z3_coef0,z3_degree, show_values=False)

# Anchors

## Setting Up

In [36]:
from __future__ import print_function
import sys
import sklearn
import sklearn.ensemble
from anchor import utils
from anchor import anchor_tabular

Generating Target Set with Reject Class == 0

In [37]:
def generate_ro_target_set(target_set,rejected_indexes):
    target_set[rejected_indexes] = 0
    return target_set

In [38]:
ro_set = generate_ro_target_set(y_train,rejected_indexes)
print(np.unique(ro_set))

[0 1]


In [39]:
feature_list = []
for i in range(0,len(X_train[0])):
    feature_list.append(str(i))
feature_list = np.array(feature_list)

In [40]:
explainer = anchor_tabular.AnchorTabularExplainer(
    [-1,0,1],
    feature_list,
    X_train)

In [41]:
def svm_decfun(data,classifier=clf):
    data = np.atleast_2d(data)
    return ((classifier.dual_coef_ @ classifier.support_vectors_) @ data.T + classifier.intercept_)[0][0]

In [42]:
def svm_decfun_class(data,classifier=clf,Threshold_1=T1,Threshold_2=T2):
    if svm_decfun(data) >= Threshold_1:
        return np.array([2]) #class 1, since [-1, 0, 1]
    elif svm_decfun(data) < Threshold_2:
        return np.array([0]) #class -1
    else:
        return np.array([1]) #class 0

### Anchors Explanation & Z3 Validation

In [50]:
def anchors_to_z3_explanation(explainer, X, t1, t2, z3_coef, z3_sup_vec, z3_X, z3_intercept, indexes,
                              z3_gamma=None, z3_coef0=None, z3_degree=None,min=0,max=1,positive=False,negative=False,rejected=False):
    print('Started')
    sat_var = 0
    unsat_var = 0
    np.random.seed(1)
    solver = Solver()
    feature_sizes = []
    anchors_time = []
    if rejected:
        print("Declared: Rejected Instances")
        solver.add(Or(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t1,
                      ((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t2))
    elif positive:
        print("Declared: Positive Instances")
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t1)
    elif negative:
        print("Declared: Negative Instances")
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t2)
    else:
        print("WARNING: Must declare if are positive,negative or rejected instances!")
        return None
    for j in range(0, len(z3_X)):
        solver.add(z3_X[j] >= min)
        solver.add(z3_X[j] <= max)
    solver.push()
    for idx in indexes:
        print('Prediction: ', explainer.class_names[svm_decfun_class(X[idx])[0]])
        start = time.perf_counter()  
        exp = explainer.explain_instance(X[idx], svm_decfun_class, threshold=1)
        anchors_time.append(time.perf_counter() - start)
        #print(exp.names())
        print('Index = ', idx)
        feature_sizes.append(len(np.unique((exp.features()))))
        for feature in np.unique(exp.features()):       
            solver.add(z3_X[feature] == X[idx][feature])
            #print("Added restrictions: ",z3_X[feature],X[idx][feature])
        
        print(solver.check(), ' for ', idx,'\n')
        if solver.check() == sat:
            model = solver.model()
            print(model)
            sat_var +=1
        else:
            unsat_var +=1
        
        print('\n---------------\n')
        solver.pop()
        solver.push()
    print("Sat = ",sat_var,"\nUnsat = ",unsat_var)
    return sat_var,unsat_var,feature_sizes,anchors_time

In [51]:
rejected_sat_var = 0
rejected_unsat_var = 0
positive_sat_var = 0
positive_unsat_var = 0
negative_sat_var = 0
negative_unsat_var = 0
rejected_feature_sizes = []
positive_feature_sizes = []
negative_feature_sizes = []
r_time = []
p_time = []
n_time  = []

In [52]:
if len(rejected_indexes)!=0:
    if len(rejected_indexes)>=50:
        rejected_sat_var, rejected_unsat_var, rejected_feature_sizes,r_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,rejected_indexes[0:50],rejected=True)
    else:
        rejected_sat_var, rejected_unsat_var, rejected_feature_sizes,r_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,rejected_indexes,rejected=True)

Started
Declared: Rejected Instances
Prediction:  0
Index =  1
sat  for  1 

[x__5 = 3479981378026071/100000000000000000,
 x__0 = 0,
 x__3 = 0,
 x__4 = 149459732234333670765732468642607183/186266294227188140000000000000000000,
 x__2 = 0,
 x__1 = 0]

---------------

Prediction:  0
Index =  5
sat  for  5 

[x__5 = 4434357541899441/100000000000000000,
 x__0 = 5599537434753682195595898831011669/10849627560521412000000000000000000,
 x__3 = 0,
 x__4 = 16141759776536313/50000000000000000,
 x__2 = 1,
 x__1 = 0]

---------------

Prediction:  0
Index =  6
sat  for  6 

[x__5 = 20879888268156423/1000000000000000000,
 x__0 = 69222463597768294122783655663403707/108496275605214120000000000000000000,
 x__3 = 0,
 x__4 = 16141759776536313/50000000000000000,
 x__2 = 1,
 x__1 = 0]

---------------

Prediction:  0
Index =  7
sat  for  7 

[x__5 = 20879888268156423/1000000000000000000,
 x__0 = 6533985102420857/50000000000000000,
 x__3 = 0,
 x__4 = 115757606779249790600466416293012639/93133147113594070000

In [53]:
if len(positive_indexes)!=0:
    if len(positive_indexes)>=50:
        positive_sat_var, positive_unsat_var, positive_feature_sizes,p_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,positive_indexes[0:50],positive=True)
    else:
        positive_sat_var, positive_unsat_var, positive_feature_sizes,p_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,positive_indexes,positive=True)

Started
Declared: Positive Instances
Prediction:  1
Index =  0
sat  for  0 

[x__5 = 25919459962756053/100000000000000000,
 x__0 = 21222067039106143/100000000000000000,
 x__3 = 0,
 x__4 = 147371986553019068002487416296481349/186266294227188140000000000000000000,
 x__2 = 0,
 x__1 = 0]

---------------

Prediction:  1
Index =  2
sat  for  2 

[x__5 = 9174813780260707/50000000000000000,
 x__0 = 21222067039106143/100000000000000000,
 x__3 = 0,
 x__4 = 67285026871486801422322599256508351/93133147113594070000000000000000000,
 x__2 = 0,
 x__1 = 0]

---------------

Prediction:  1
Index =  3
sat  for  3 

[x__5 = 9174813780260707/50000000000000000,
 x__0 = 21222067039106143/100000000000000000,
 x__3 = 20544692737430167/100000000000000000,
 x__4 = 33793409482778624061923775397147491/46566573556797035000000000000000000,
 x__2 = 0,
 x__1 = 284171322160149/4000000000000000]

---------------

Prediction:  1
Index =  4
sat  for  4 

[x__5 = 9174813780260707/50000000000000000,
 x__0 = 293381732803265

In [54]:
if len(negative_indexes)!=0:
    if len(negative_indexes)>=50:
        negative_sat_var, negative_unsat_var, negative_feature_sizes,n_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,negative_indexes[0:50],negative=True)
    else:
        negative_sat_var, negative_unsat_var, negative_feature_sizes,n_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,negative_indexes,negative=True)

## Calculating Metrics

In [55]:
soma = 0
tamanho = 0
if rejected_elapsed_time != None and len(rejected_elapsed_time) >0:
    soma += sum(rejected_elapsed_time)
    tamanho += len(rejected_elapsed_time)
    print("Tempo Médio Z3 Rejeitados = ",sum(rejected_elapsed_time)/len(rejected_elapsed_time))
if positive_elapsed_time != None and len(positive_elapsed_time) >0:
    soma += sum(positive_elapsed_time)
    tamanho += len(positive_elapsed_time)
    print("Tempo Médio Z3 Positivos = ",sum(positive_elapsed_time)/len(positive_elapsed_time))
if negative_elapsed_time != None and len(negative_elapsed_time) >0:
    soma += sum(negative_elapsed_time)
    tamanho += len(negative_elapsed_time)
    print("Tempo Médio Z3 Negativos = ",sum(negative_elapsed_time)/len(negative_elapsed_time)) 
if tamanho != 0:
    print("Tamanho Médio Total: ",soma/tamanho,'sec')
    print("Soma: ",soma," Tamanho: ",tamanho)

Tempo Médio Z3 Rejeitados =  0.01049525200000005
Tempo Médio Z3 Positivos =  0.008401163999999994
Tamanho Médio Total:  0.009448208000000022 sec
Soma:  0.9448208000000022  Tamanho:  100


In [56]:
soma = 0
tamanho = 0
if r_time != None and len(r_time)>0:
    soma += sum(r_time)
    tamanho += len(r_time)
    print("Tempo Médio Anchors Rejeitados = ",sum(r_time)/len(r_time))
if p_time != None and len(p_time)>0:
    soma += sum(p_time)
    tamanho += len(p_time)
    print("Tempo Médio Anchors Positivos = ",sum(p_time)/len(p_time))
if n_time != None and len(n_time)>0:
    soma += sum(n_time)
    tamanho += len(n_time)
    print("Tempo Médio Anchors Negativos = ",sum(n_time)/len(n_time))      
if tamanho != 0:
    print("Tamanho Médio Total: ",soma/tamanho,'sec')
    print("Soma: ",soma," Tamanho: ",tamanho)

Tempo Médio Anchors Rejeitados =  0.252087554
Tempo Médio Anchors Positivos =  0.583033004
Tamanho Médio Total:  0.417560279 sec
Soma:  41.7560279  Tamanho:  100


In [57]:
soma = 0
tamanho = 0
if len(rejected_linear_relevant)>0:  
    tamanho += len(rejected_linear_relevant)
    tamanho_rejeitado = 0
    for x in rejected_linear_relevant:
        soma += len(x)
        tamanho_rejeitado += len(x)
    print("Z3 Tamanho Rejeitado: ",tamanho_rejeitado,"Tamanho Médio Rejeitado:",tamanho_rejeitado/len(rejected_linear_relevant))
if len(positive_linear_relevant)>0:   
    tamanho += len(positive_linear_relevant)
    tamanho_positivo = 0
    for x in positive_linear_relevant:
        soma += len(x)
        tamanho_positivo += len(x)
    print("Z3 Tamanho Positivo: ",tamanho_positivo,"Tamanho Médio Positivo:",tamanho_positivo/len(positive_linear_relevant))
if len(negative_linear_relevant)>0:
    tamanho += len(negative_linear_relevant)
    tamanho_negativo = 0
    for x in negative_linear_relevant:
        soma += len(x)
        tamanho_negativo += len(x)
    print("Z3 Tamanho Negativo: ",tamanho_negativo,"Tamanho Médio Negativo:",tamanho_negativo/len(negative_linear_relevant))
print("Z3 Tamanho Médio ",soma/tamanho)
print("Soma: ",soma,"Tamanho: ",tamanho)

Z3 Tamanho Rejeitado:  211 Tamanho Médio Rejeitado: 4.22
Z3 Tamanho Positivo:  173 Tamanho Médio Positivo: 3.46
Z3 Tamanho Médio  3.84
Soma:  384 Tamanho:  100


In [58]:
if len(rejected_feature_sizes) > 0 :
    print("Anchors Tamanho Rejeitado: ",sum(rejected_feature_sizes),"Tamanho Médio Rejeitado",sum(rejected_feature_sizes)/len(rejected_feature_sizes))
if len(positive_feature_sizes) > 0 :
    print("Anchors Tamanho Positivo: ",sum(positive_feature_sizes),"Tamanho Médio Positivo",sum(positive_feature_sizes)/len(positive_feature_sizes))
if len(negative_feature_sizes) > 0 :
    print("Anchors Tamanho Negativo: ",sum(negative_feature_sizes),"Tamanho Médio Negativo",sum(negative_feature_sizes)/len(negative_feature_sizes))
soma = sum(rejected_feature_sizes)+sum(positive_feature_sizes)+sum(negative_feature_sizes)
soma_len = len(rejected_feature_sizes) + len(positive_feature_sizes) + len(negative_feature_sizes)
print("Anchors Tamanho Médio: ",soma/soma_len)
print("Soma: ",soma,"Tamanho: ",soma_len)

Anchors Tamanho Rejeitado:  88 Tamanho Médio Rejeitado 1.76
Anchors Tamanho Positivo:  184 Tamanho Médio Positivo 3.68
Anchors Tamanho Médio:  2.72
Soma:  272 Tamanho:  100


In [59]:
print("          SAT/UNSAT")
print("Rejected: ",rejected_sat_var,"|",rejected_unsat_var)
print("Positive: ",positive_sat_var,"|",positive_unsat_var)
print("Negative: ",negative_sat_var,"|",negative_unsat_var)
soma_sat = rejected_sat_var + positive_sat_var + negative_sat_var
soma_unsat = rejected_unsat_var + positive_unsat_var + negative_unsat_var
print("Total:    ",soma_sat,"|",soma_unsat," of ",soma_sat+soma_unsat)

          SAT/UNSAT
Rejected:  50 | 0
Positive:  30 | 20
Negative:  0 | 0
Total:     80 | 20  of  100
